<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/Text_Classification_Bank_Complaints_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!ls

sample_data


In [32]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!ls '/content/drive/My Drive/Colab Notebooks/Models/'

complaints.booster  Image  tfvectorizer.pkl


In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import nltk
import joblib
import xgboost as xgb

In [36]:
df=pd.read_csv('https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset//NLP/consumer_compliants.zip?raw=true',compression='zip',sep=',',quotechar='"')

In [38]:
df=df.sample(10000)

In [39]:
complanints_df=df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints','Product':'product','Company':'company'})

In [40]:
target= {'Debt collection':0,'Credit card or prepaid card':1,'Mortgage':2,'Checking or savings account':3,'Student loan':4,'Vehicle loan or lease':5}

In [41]:
pd.set_option('display.max_colwidth',-1)
complanints_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,complaints,product,company
52561,Today I checked my credit and discovered that this company has reported that I failed to pay them for services rendered. Ive check my records and I have never done business with this company and never entered in a contract to do so.,Debt collection,"Automated Collection Services, Inc."
54321,Payment on may Valley Bank credit card is due tomorrow XXXX XXXX. So my beloved wife XXXX and I drove to the local branch in XXXX a mile or so from our home. BUT THE DRIVE IN WAS CLOSED AS WAS THE BRANCH ITSELF. So I mailed my check at the XXXX Post Office for hoped for delivery to the XXXX branch of the bank on XXXX XXXX! \n\nBut I fear that the Valley Bank will claim it did not receive my check on XXXX XXXX and so charge me a sky-high late fee which would be totally unjust as its branch and its branch drive in were closed as aforementioned! \n\nI SHALL NOT PAY ANY LATE FEE OR INTEREST NOR MAY THE VALLEY BANK REPORT ME TO THE CREDIT REPORTING BUREAUS! \n\nThe branch should have been open even during the coronavirus pandemic! Where does Valley Bank think customers are to bank? ON THE MOON?,Credit card or prepaid card,U.S. BANCORP
22986,"I was contacted by Credit Collections Services on XX/XX/19 in reference to an attempt to collect a debt owed to XXXX XXXX in the amount of XXXX. In response to this letter I set up a payment plan breaking the above amount in to three payments to be paid via check with the first payment to received by XX/XX/19. I received an official letter from this company confirming the agreed payment plan in the mail dated for XX/XX/19. I mailed a check to credit collections solutions via priority mail on XX/XX/19 made out to XXXX XXXX, which is 9 days prior to the due date. I received a letter from credit collections services on XX/XX/19 stating that they have not received a payment as of XX/XX/19, and that they are canceling my payment plan. However in the same letter received on XX/XX/19 it clearly shows that the debt owed has reduced from XXXX to XXXX, which clearly proves that they did receive a payment. There would be no reason for me to make another payment until XX/XX/19 based on the payment agreement made with Credit Collection Services. Upon talking to one of their representatives on XX/XX/19, she insinuated that this is the companies standard practice, to send letters out to people stating that payment arrangements are being canceled even though they are actually adhering to the agreed payment plan. This person I spoke to on the phone was able to verify that they received a payment so there was no reason for this letter. Last but definitely not least, this company reported this false information on my credit report stating that they have not received a payment for XXXX when in fact per our payment agreement a payment would not be due until XX/XX/19. I find this very unprofessional and a form of retaliation.",Debt collection,"CCS Financial Services, Inc."
2729,"Per response to my previous CFPB complaint of garnishing me without proper notification, Midland Funding indicated that their attorney, XXXX XXXX, would be refunding me for the money collected from my paychecks, including poundage fees. \n\nThe garnishment has since been released and of the {$5900.00} that was garnished from me in XXXX and XXXX I have received a refund of {$4700.00} from XXXX and XXXX checks in the amount of {$550.00} each from the courts. \n\nThis means, as of today XXXX, I am still due a refund of {$110.00} in fees withheld by the courts.",Debt collection,ENCORE CAPITAL GROUP INC.
26918,I disputed a charge and chase refused to contract the merchant on my behalf.,Credit card or prepaid card,JPMORGAN CHASE & CO.
...,...,...,...
11711,"I contacted CONVERGENT OUTSOURCING and indicated I needed to know how the cable company ( XXXX ) had come to the amount they were trying to collect. ( {$400.00} ) I asked for them to send me a debt validation to include how they had come up to the amount. I indicated I had 

In [42]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
#stemmer= PorterStemmer()
stemmer= nltk.stem.SnowballStemmer('english')
stop_words=set(nltk.corpus.stopwords.words('english'))

In [44]:
def tokenize(text):
  tokens=[word for word in nltk.word_tokenize(text)if(len(word) >3 and len(word.strip('Xx/'))>2 and len(re.sub('\d+','',word.strip('Xx/'))))]
  tokens=map(str.lower,tokens)
  stems=[stemmer.stem(item) for item in tokens if (item not in stop_words)]
  return stems

In [45]:
!ls '/content/drive/My Drive/Colab Notebooks/Models/'

complaints.booster  Image  tfvectorizer.pkl


In [46]:
tfvectorizer= joblib.load('/content/drive/My Drive/Colab Notebooks/Models/tfvectorizer.pkl')

In [47]:
test_transformed= tfvectorizer.transform(complanints_df['complaints'])

In [48]:
tfvectorizer.get_feature_names()

['0.00',
 '100.00',
 '1000.00',
 '200.00',
 '2000.00',
 '300.00',
 '500.00',
 'abil',
 'abl',
 'absolut',
 'abus',
 'accept',
 'access',
 'accord',
 'account',
 'account account',
 'account balanc',
 'account bank',
 'account call',
 'account close',
 'account credit',
 'account inform',
 'account number',
 'account open',
 'account report',
 'account would',
 'accru',
 'acct',
 'accur',
 'acknowledg',
 'action',
 'activ',
 'actual',
 'ad',
 'addit',
 'address',
 'adjust',
 'admit',
 'advanc',
 'advantag',
 'advertis',
 'advis',
 'affect',
 'affidavit',
 'afford',
 'agenc',
 'agent',
 'agre',
 'agreement',
 'alert',
 'alleg',
 'alleg debt',
 'alli',
 'allow',
 'almost',
 'along',
 'alreadi',
 'also',
 'although',
 'alway',
 'america',
 'american',
 'american express',
 'amex',
 'amount',
 'amount owe',
 'and/or',
 'annual',
 'anoth',
 'answer',
 'anyon',
 'anyth',
 'apart',
 'appar',
 'appeal',
 'appear',
 'appli',
 'applic',
 'apprais',
 'appreci',
 'approv',
 'approxim',
 'around',
 

In [49]:
test_transformed.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [50]:
!ls '/content/drive/My Drive/Colab Notebooks/Models/'

complaints.booster  Image  tfvectorizer.pkl


In [53]:
xgb_clf=xgb.Booster({'nthread':4})
xgb_clf.load_model('/content/drive/My Drive/Colab Notebooks/Models/complaints.booster')

In [54]:
predictions=xgb_clf.predict(xgb.DMatrix(test_transformed))

In [55]:
predictions

array([[0.67065537, 0.14616476, 0.05340694, 0.07117446, 0.02455008,
        0.03404839],
       [0.02467364, 0.89115614, 0.00706848, 0.05890116, 0.00717311,
        0.01102746],
       [0.96417445, 0.00832843, 0.00686974, 0.00507148, 0.00438729,
        0.01116861],
       ...,
       [0.5114865 , 0.18507926, 0.10065143, 0.11384927, 0.03731014,
        0.05162339],
       [0.96390545, 0.01163929, 0.00813391, 0.00697842, 0.00398525,
        0.00535764],
       [0.4175762 , 0.21068154, 0.09252395, 0.05238758, 0.1037096 ,
        0.12312114]], dtype=float32)

In [56]:
test_transformed=tfvectorizer.transform(["I have a federal student loan and there is supposed to be a zero percent interest rate"])

In [57]:
final_class= xgb_clf.predict(xgb.DMatrix(test_transformed))

In [58]:
final_class

array([[0.06598752, 0.01048827, 0.02473602, 0.00983872, 0.876513  ,
        0.01243649]], dtype=float32)

In [70]:
target.get(final_class.argmax(axis=1)[0])

In [71]:
from sklearn.metrics import classification_report,confusion_matrix

In [72]:
reverse_target={v:k for k,v in target.items()}

In [73]:
reverse_target

{0: 'Debt collection',
 1: 'Credit card or prepaid card',
 2: 'Mortgage',
 3: 'Checking or savings account',
 4: 'Student loan',
 5: 'Vehicle loan or lease'}

In [ ]:
cm = confusion_matrix(complanints_df['product'].map(reverse_target), predictions.argmax(axis=1))
print(cm)

In [77]:
print('classification report\n',classification_report(complanints_df['product'].map(reverse_target),predictions.argmax(axis=1)))

ValueError: ignored